# Análise de projetos de infraestrutura com investimento federal no Distrito Federal


Teste avaliativo para vaga de bolsista em engenharia/análise de dados (LabLivre). Ver [material de referência](https://docs.google.com/document/d/1WWBnAodJoDo40WZ6ysI5cGx90KFYAp7GTFLrWPjmMFE/edit?tab=t.0)

Fonte de dados: [obrasgov](https://api.obrasgov.gestao.gov.br/obrasgov/api/swagger-ui/index.html#/Projeto%20De%20Investimento/buscarPorFiltro)

Relatório detalhado de tratamento de dados com visualizações. Para análises acessíveis ver a conclusão. Em resumo:

-   ... TODO

-   TOC

Observação: IA (Cursor) foi usado para a produção deste relatório, principalmente para agilizar a produção de gráficos.


In [593]:
import json
import time
import requests
import glob
import sqlite3

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
import pingouin as pg
import statsmodels.api as sm
import hdbscan
import umap.umap_ as umap

# cores do lablivre
palette = ["#412355", "#F2701C", "#18CEE6"]

## 1. Extração dos dados


In [594]:
# script de coleta de dados foi executado separadamente


def get_data(page: int) -> dict:
    url = "https://api.obrasgov.gestao.gov.br/obrasgov/api/projeto-investimento"
    params = {"uf": "DF", "pagina": page, "tamanhoDaPagina": 100}
    headers = {"accept": "*/*"}

    response = requests.get(url, params=params, headers=headers)

    if response.status_code != 200:
        raise Exception(f"Failed to get data: {response.status_code}")

    return response.json()


def main():
    # inelegante mas eficiente. certamente não tem 10000 itens
    for page in range(100):
        response = get_data(page)
        print(f"Page {page} processed")

        # salvar dados brutos primeiro para não depender da api caso ocorram problemas
        with open(f"data/data-{page}.json", "w", encoding="utf-8") as f:
            json.dump(response["content"], f, indent=4)
        print(f"Data saved to data/data-{page}.json")

        time.sleep(1)

        page += 1


# if __name__ == "__main__":
#     main()

## 2. Tratamento de dados


In [595]:
def load_json_files() -> list[dict]:
    json_files = glob.glob("data/data-*.json")
    all_records = []

    for file in json_files:
        with open(file, "r", encoding="utf-8") as f:
            records = json.load(f)
            all_records.extend(records)

    return all_records


df = pd.DataFrame(load_json_files())

df.info(memory_usage="deep")

In [596]:
nested_cols = [
    "tomadores",
    "executores",
    "repassadores",
    "eixos",
    "tipos",
    "subTipos",
    "fontesDeRecurso",
]

### 2.1 Duplicatas


In [597]:
from analyse_duplicates import check_duplicates, analyze_false_duplicates

# existem duplicatas que só diferem no conteúdo das listas. não quero entediar você com esse código gerado por IA então movi para outro arquivo

check_duplicates(df)
analyze_false_duplicates(df, "results/duplicate_records_report")

In [598]:
# após investigação detalhada, todas as 'falsas duplicatas' se parecem com isso:

# Linha 133: [{'id': 90, 'descricao': 'Preservação do Patrimônio', 'idTipo': 5}] (5 = Administrativo)
# Linha 491: [{'id': 90, 'descricao': 'Preservação do Patrimônio', 'idTipo': 43}] (43 = Cultura)

# para mais exemplos, consulte o relatório (`results/duplicate_records_report.txt`).
# não tenho certeza do motivo disso; talvez os rótulos sejam ambíguos para as pessoas que preenchem algum formulário.
# mas isso parece indicar que são, de fato, duplicatas. então serão removidas.

df.duplicated(subset=["idUnico"]).sum()
df.drop_duplicates(subset=["idUnico"], inplace=True)

In [599]:
pd.set_option("display.max_columns", None)
df.head()

### 2.2 Valores ausentes


In [600]:
# remover ceps preenchidos com 1 ou espaço.
df["cep"] = df["cep"].str.replace(r"^1$", "", regex=True)
df["cep"] = df["cep"].str.replace(r"\s+", "", regex=True).replace("", None)
df["cep"] = df["cep"].str.replace("-", "")
df["cep"] = df["cep"].str.replace(".", "")

# remover ceps com menos de 8 dígitos
df["cep"] = df["cep"].apply(lambda x: None if pd.isna(x) or len(str(x)) < 8 else x)

df["cep"].nunique()

In [601]:
# remover enderecos preenchidos com 1 ou espaço.
df["endereco"] = df["endereco"].str.strip()
df["endereco"] = df["endereco"].str.replace(r"^1$", "", regex=True)
df["endereco"] = df["endereco"].str.replace(r"^\s+$", "", regex=True).replace("", None)
df["endereco"].nunique()

In [602]:
object_cols = df.drop(columns=nested_cols, axis=1)
info_df = pd.DataFrame(
    {
        "Unique Values": object_cols.nunique(),
        "% Unique": (object_cols.nunique() / object_cols.count() * 100).round(1),
        "NaN Values": object_cols.isna().sum(),
        "% NaN": (object_cols.isna().sum() / len(object_cols) * 100).round(1),
    }
)
info_df[info_df["% NaN"] > 1].sort_values(by="% NaN", ascending=False)

Há muitos valores ausentes. Não entendo o motivo disso, mas não me parece um erro. Muitos desses são datas que suspeito serem futuras, ou campos do tipo 'outras observações'. Mais notável é a ausência de `qdtEmpregosGerados`, `populacaoBeneficiada`, `cep` e `endereco`. Não tiraria conclusões precipitadas sem saber mais.

Também não vejo necessidade de fazer imputação aqui. Então todos permanecem como estão.


### 2.3 Tipagem


In [603]:
# usar isso em cada coluna para verificar o tipo da variável
df["isModeladaPorBim"].value_counts(dropna=False)

In [604]:
df["isModeladaPorBim"] = df["isModeladaPorBim"].astype("boolean")

In [605]:
# variaveis categoricas
df["natureza"] = df["natureza"].astype("category")
df["situacao"] = df["situacao"].astype("category")
df["especie"] = df["especie"].astype("category")
df["uf"] = df["uf"].astype("category")

In [606]:
# variaveis de data
# verifiquei manualmente os registros para erros de formatação mas não encontrei problemas

date_cols = [
    "dataInicialPrevista",
    "dataFinalPrevista",
    "dataInicialEfetiva",
    "dataFinalEfetiva",
    "dataCadastro",
    "dataSituacao",
]

for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors="coerce")

In [607]:
# alguns registros tem strings em vez de ints. esses registros também têm outros problemas; veja abaixo.
# isso coage strings para null
# converte para float em vez de int para permitir valores nulos
df["qdtEmpregosGerados"] = pd.to_numeric(df["qdtEmpregosGerados"], errors="coerce")
df["populacaoBeneficiada"] = pd.to_numeric(df["populacaoBeneficiada"], errors="coerce")

In [608]:
df.info(memory_usage="deep")

### 2.4 Colunas aninhadas


In [609]:
def normalize_nested_column(
    df: pd.DataFrame, id_col: str, nested_col: str
) -> pd.DataFrame:
    def convert_dict_values_to_str(x):
        # pd.json_normalize(df['tomadores'].explode()) converte o id para float, o que pode causar problemas devido à imprecisão de ponto flutuante mesmo se convertido de volta para int/str. por isso, preciso converter os valores para strings primeiro, o que requer um código um pouco confuso:

        if not isinstance(x, list):
            return x
        return [{key: str(value) for key, value in item.items()} for item in x]

    df[nested_col] = df[nested_col].apply(convert_dict_values_to_str)
    exploded = df[[id_col, nested_col]].explode(nested_col)
    exploded = exploded.dropna(subset=[nested_col])
    normalized = pd.json_normalize(exploded[nested_col])
    normalized[id_col] = exploded[id_col].values
    return normalized


def create_entity_table_from_junction_table(
    old_df: pd.DataFrame, prev_index: str, actual_index: str
) -> pd.DataFrame:
    new_df = old_df.dropna(subset=[prev_index])
    new_df.drop_duplicates(subset=[actual_index], inplace=True)
    new_df.set_index(actual_index, inplace=True)
    new_df.drop(prev_index, axis=1, inplace=True)
    return new_df

In [610]:
# reuni essas tabelas em uma só, pois são todas referentes a instituicoes e usam o mesmo sistema de códigos.

projeto_tomadores_df = normalize_nested_column(df, "idUnico", "tomadores")
projeto_executores_df = normalize_nested_column(df, "idUnico", "executores")
projeto_repassadores_df = normalize_nested_column(df, "idUnico", "repassadores")

tomadores_df = create_entity_table_from_junction_table(
    projeto_tomadores_df, "idUnico", "codigo"
)
executores_df = create_entity_table_from_junction_table(
    projeto_executores_df, "idUnico", "codigo"
)
repassadores_df = create_entity_table_from_junction_table(
    projeto_repassadores_df, "idUnico", "codigo"
)

instituicoes_df = pd.concat([tomadores_df, executores_df, repassadores_df])

# deduplicate indices
instituicoes_df = instituicoes_df.loc[~instituicoes_df.index.duplicated(keep="first")]

instituicoes_df.head(1)

In [611]:
projeto_eixos_df = normalize_nested_column(df, "idUnico", "eixos")

eixos_df = create_entity_table_from_junction_table(projeto_eixos_df, "idUnico", "id")

eixos_df.head(1)

In [612]:
projeto_tipos_df = normalize_nested_column(df, "idUnico", "tipos")

tipos_df = create_entity_table_from_junction_table(projeto_tipos_df, "idUnico", "id")

tipos_df.head(1)

In [613]:
projeto_subtipos_df = normalize_nested_column(df, "idUnico", "subTipos")

subtipos_df = create_entity_table_from_junction_table(
    projeto_subtipos_df, "idUnico", "id"
)

subtipos_df.head(1)

In [614]:
# nesse caso é one to many e nao many to many, entao nao precisa da tabela intermediaria. mas reutiliza o codigo de qualquer forma

fontes_de_recurso_df = normalize_nested_column(
    df, "idUnico", "fontesDeRecurso"
).reset_index(drop=True)

# nesse caso estava certo o float
fontes_de_recurso_df["valorInvestimentoPrevisto"] = pd.to_numeric(
    fontes_de_recurso_df["valorInvestimentoPrevisto"], errors="coerce"
)

fontes_de_recurso_df.head(1)

In [615]:
df.drop(columns=nested_cols, inplace=True)

### 2.X Limpeza de dados


#### Encoding


In [616]:
# erro de encoding. todos parecem seguir nomes parecidos, indicando que o erro deve ser relacionado à fonte. endereço são rodovias.
# Texto em UTF-8, foi lido como Latin-1 (ISO-8859-1)
enconding_issue = df[df["descricao"].str.contains("Ã§Ã")]
print(f"Pelo menos {enconding_issue.shape[0]} projetos com erro de encoding")
df[df["descricao"].str.contains("Ã§Ã")].head(1)

In [617]:
def fix_encoding(item: any) -> any:
    if isinstance(item, str):
        try:
            return item.encode("latin-1").decode("utf-8")
        except:
            return item
    return item


for col in df.select_dtypes(include=["object"]).columns:
    df[col] = df[col].apply(fix_encoding)

#### Registros incorretos devido a testes de integração


In [618]:
# há um sujeito chamado Ronald Alves Vieira e ele está testando em produção
test_df = df[df["nome"].str.contains("Ronald", case=False, na=False)]
test_df.head()

In [619]:
# é possível que hajam mais casos que não encontrei
# é possível que essa limpeza remova dados reais


antes = df.shape[0]

df = df[~df["nome"].str.contains("Ronald", case=False, na=False)]
df = df[~df["nome"].str.contains("Teste", case=False, na=False)]

depois = df.shape[0]

print(
    f"Removidos {antes - depois} projetos por serem registros falsos (efeitos colaterais de testes de integração)"
)

### 2.5 Carregamento de dados


In [620]:
with sqlite3.connect("projeto_investimento.db") as con:
    # main
    df.to_sql("projeto_investimento", con, if_exists="replace")

    # entity tables
    instituicoes_df.to_sql("instituicoes", con, if_exists="replace")
    eixos_df.to_sql("eixos", con, if_exists="replace")
    tipos_df.to_sql("tipos", con, if_exists="replace")
    subtipos_df.to_sql("subtipos", con, if_exists="replace")
    fontes_de_recurso_df.to_sql("fontes_de_recurso", con, if_exists="replace")

    # junction tables
    projeto_tomadores_df.to_sql("projeto_tomadores", con, if_exists="replace")
    projeto_executores_df.to_sql("projeto_executores", con, if_exists="replace")
    projeto_repassadores_df.to_sql("projeto_repassadores", con, if_exists="replace")
    projeto_eixos_df.to_sql("projeto_eixos", con, if_exists="replace")
    projeto_tipos_df.to_sql("projeto_tipos", con, if_exists="replace")
    projeto_subtipos_df.to_sql("projeto_subtipos", con, if_exists="replace")

In [621]:
# with sqlite3.connect("projeto_investimento.db") as con:
#     df = pd.read_sql_query("SELECT * FROM projeto_investimento", con)

### 2.5 Engenharia de características


Criação de novas colunas para fins de análise

-   `textoTotal`: concatenação de dados textuais de um registro para processamento de linguagem natural (NLP)
-   `investimentoTotal`: consolidação de `fontesDeRecurso` para análise quantitativa
-   `investimentoFaixa`: versão categórica de `investimentoTotal` por faixas (por exemplo, de 100 a 1000, de 1,000 a 10,000, etc.)
-   `isInvestimentoSimbolico`: investimentos com valores menores que 1 real; assume-se que possuem valor simbólico
-   `duracaoPrevista`: calcular duração prevista de projetos (via `dataInicialPrevista`, `dataFinalPrevista`)
-   `lat`, `lon`: obtenção de dados geográficos (via `cep`) usando geocoding para visualização gráfica
-   calcular


In [622]:
# textoTotal

text_cols = [
    "nome",
    "endereco",
    "descricao",
    "funcaoSocial",
    "metaGlobal",
    "especie",
    "natureza",
    "naturezaOutras",
    "situacao",
    "descPlanoNacionalPoliticaVinculado",
    "descPopulacaoBeneficiada",
    "observacoesPertinentes",
]


def make_text_total(row):
    text_total = []
    for x in row:
        if pd.notna(x):
            text_total.append(str(x))
    return "\n\n".join(text_total)


df["textoTotal"] = df[text_cols].apply(make_text_total, axis=1)

plt.figure(figsize=(10, 6))
sns.histplot(data=df["textoTotal"].str.len(), bins=30)
plt.title("Distribution of Text Lengths")
plt.xlabel("Number of Characters")
plt.ylabel("Count")
plt.show()

In [623]:
# investimentoTotal

valor_por_id = (
    fontes_de_recurso_df.groupby("idUnico")["valorInvestimentoPrevisto"]
    .sum()
    .reset_index()
)

valor_por_id.rename(
    columns={"valorInvestimentoPrevisto": "investimentoTotal"}, inplace=True
)

df = df.merge(valor_por_id, on="idUnico", how="left", validate="one_to_one")

In [624]:
# investimentoFaixa

bins = [
    0,
    1,
    10,
    100,
    1_000,
    10_000,
    100_000,
    1_000_000,
    10_000_000,
    100_000_000,
    1_000_000_000,
    np.inf,
]
labels = [
    "<1",
    "1-10",
    "10-100",
    "100-1k",
    "1k-10k",
    "10k-100k",
    "100k-1M",
    "1M-10M",
    "10M-100M",
    "100M-1B",
    ">1B",
]

df["investimentoFaixa"] = pd.cut(df["investimentoTotal"], bins=bins, labels=labels)

In [625]:
faixa_counts = df["investimentoFaixa"].value_counts().reset_index()
faixa_counts.columns = ["investimentoFaixa", "count"]

plt.figure(figsize=(10, 6))
ax = sns.barplot(
    data=faixa_counts,
    x="investimentoFaixa",
    y="count",
    hue="investimentoFaixa",
    palette=[palette[0] for _ in range(len(faixa_counts))],
)

# custom palette creates multiple containers
for container in ax.containers:
    ax.bar_label(container)  # type: ignore

plt.xlabel("Faixa de Investimento (R$)")
plt.ylabel("Projetos")
plt.title("Projetos por Faixa de Investimento")
plt.tight_layout()
plt.show()

In [626]:
# isInvestimentoSimbolico

df["isInvestimentoSimbolico"] = df["investimentoTotal"] <= 1
df["isInvestimentoSimbolico"].value_counts()

In [627]:
# duracaoPrevista
df["duracaoPrevista"] = df["dataFinalPrevista"] - df["dataInicialPrevista"]

In [628]:
# lat, lon

# todo

## 3. Análise


### 3.1 Informações básicas

**Colunas:**

-   **texto**

    -   dados básicos: `idUnico`, `nome`, `descricao`, `funcaoSocial`, `metaGlobal`
    -   descrições opcionais: `naturezaOutras`, `descPlanoNacionalPoliticaVinculado`, `descPopulacaoBeneficiada`, `observacoesPertinentes`
    -   criada: `textoTotal`, `embeddings`, `cluster`

-   **geográfico**

    -   `cep`, `endereco`
    -   criadas: `lat_ipedf`, `lon_ipedf`, `lat_viacep`, `lon_viacep`

-   **numéricos**

    -   `qdtEmpregosGerados`
    -   `populacaoBeneficiada`
    -   criada: `investimentoTotal`

-   **datas**

    -   `dataInicialPrevista`, `dataFinalPrevista`, `dataInicialEfetiva`, `dataFinalEfetiva`, `dataCadastro`, `dataSituacao`
    -   criada: `duracaoPrevista`

-   **categorias**

    -   `natureza` (Estudo, Obra, Outros, Projeto, Projeto de Investimento em Infraestrutura)
    -   `situacao` (Cadastrada, Cancelada, Concluída, Em execução, Inacabada, Inativada, Paralisada)
    -   `especie` (Ampliação, Construção, Fabricação, Máquinas e Equipamentos, Recuperação, Reforma)
    -   `uf` (DF)
    -   `isModeladaPorBim` (boolean)
    -   criada: `investimentoFaixa` (boolean)
    -   criada: `isInvestimentoSimbolico` (boolean)

-   **colunas aninhadas** (removidas após normalização)
    -   `tomadores`, `executores`, `repassadores`, `eixos`, `tipos`, `subTipos`, `fontesDeRecurso`

**Registros:** 712 (após deduplicação de 834 registros originais)

**Valores ausentes mais significativos:**

-   `dataFinalEfetiva`: 707 nulos (99.3%)
-   `dataInicialEfetiva`: 690 nulos (96.9%)
-   `qdtEmpregosGerados`: 677 nulos (95.1%)
-   `populacaoBeneficiada`: 677 nulos (95.1%)
-   `endereco`: 337 nulos (47.3%)
-   `cep`: 365 nulos (51.3%)
-   `descPlanoNacionalPoliticaVinculado`: 464 nulos (65.2%)
-   `isModeladaPorBim`: 216 nulos (30.3%)

As colunas aninhadas foram normalizadas e separadas em tabelas relacionadas no banco de dados SQLite, incluindo tabelas de entidades (instituições, eixos, tipos, subtipos, fontes de recurso) e tabelas de junção (projeto_tomadores, projeto_executores, projeto_repassadores, projeto_eixos, projeto_tipos, projeto_subtipos).

Tomadores, executores e repassadores partilham do mesmo sistema de códigos e portanto estão armazenados na mesma tabela.

Eixos, tipos, subtipos poderiam ser armazenados na mesma tabela com perda de normalização mas ganho em simplicidade.


### 3.2 Qualidade


Em questão de anomalias, temos:

1.  duplicados que diferem apenas no subtipo, como: dois registros com id 15 e mesma descrição "Aquisição de Máquinas e Equipamentos", mas idTipo diferentes (41 e 14).

2.  organizacoes com mais de um codigo, como: "PRESIDÊNCIA DA REPÚBLICA" (codigo 26) e "Presidencia da República" (codigo 20000)

3.  Vários campos (`cep`, `endereco`, `qdtEmpregosGerados`) preenchidos com um espaço em branco, dando a impressão que tem menos nulos do que realmente tem.

    4.  CEPs preenchidos errado (`1` e ' ')
    5.  125 registros similares com os mesmos problemas (' ' em vez de int em `qdtEmpregosGerados` e `populacaoBeneficiada`)

4.  Registro de teste (`22312.53-84`), encontrado porque é o único com dois investimentos, os dois com valor de um centavo.

5.  `descPopulacaoBeneficiada` com texto padrão

6.  `descPlanoNacionalPoliticaVinculado`: alguns usuários tratam como valor categórico e outros como texto

7.  `populacaoBeneficiada` alguns registros todos em maiusculo ('CONSTRUÇÃO DO NOVO IML DA PCDF')

8.  alguns nomes são apenas códigos (`26.782.2087.7T98.0025`)

9.  22 registros são testes e foram descartados; alguns possuem valores de investimento que poderam distorcer análises. alguns são mais obviamente testes (`40456.53-49`) do que outros (`2843.53-36`)

10. 8 projetos com exatamente 50 empregos gerados sem relação aparente entre si

    -   idUnico: 11486.53-40, 19176.53-10, 30067.53-80, 31923.53-48,9854.53-01, 91397.53-15, 91392.53-43, 31897.53-36.

11. projetos com investimento entre 1 e 10 milhões com duração prevista de zero dias.

12. `cep` preenchido com 7000000, ou com menos que 8 dígitos

Sem mais informações sobre o domínio, é dificil comentar o significado dessas questões. Mas são algo a ser investigado. Possivelmente, isso indica confusão pela parte dos usuários, ou falta de adequação entre os campos do formulário e a realidade dos projetos. De qualquer forma, uma forma de reduzir isso seria incluir validação antes de salvar no banco de dados (por exemplo, recusar o CEP '1') ou transformá-los antes de inserir (por exemplo, converter o CEP ' ' para nulo)


### 3.3 Descritiva


#### 3.3.1 Variáveis categóricas


In [629]:
df.head(5)

In [630]:
palette_bool = ["#18CEE6", "#412355", "#F2701C"]

# Convert and reorder
bim_map = {True: "Sim", False: "Não"}
bim_counts = (
    df["isModeladaPorBim"].map(bim_map).fillna("Sem dados").value_counts(dropna=False)
)

# Specify custom order
desired_order = ["Sim", "Não", "Sem dados"]
bim_counts = bim_counts.reindex(desired_order)
percentages = bim_counts / bim_counts.sum() * 100

# Create stacked bar
fig, ax = plt.subplots(figsize=(8, 2))
left = 0
colors = [palette_bool[i] for i in range(len(bim_counts))]
for i, (label, pct) in enumerate(zip(bim_counts.index, percentages)):
    ax.barh(
        0,
        pct,
        left=left,
        color=colors[i],
        label=f"{label}: {bim_counts.iloc[i]} ({pct:.1f}%)",
    )
    left += pct

ax.set_xlim(0, 100)
ax.set_yticks([])
ax.set_xlabel("Percentage")
ax.set_title("Projeto modelado usando BIM (Building Information Modeling)")
ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))
plt.tight_layout()
plt.show()

Apenas 3.5% dos projetos usou BIM, enquanto 66.2% não usou.


In [631]:
def make_barh_plot(
    df: pd.DataFrame,
    column: str,
    title: str,
    color: str,
    extra_space: float = 1,
    show_percentages: bool = False,
):
    plt.figure(figsize=(10, 6))

    # Get counts and percentages
    value_counts = df[column].value_counts(dropna=False)
    percentages = (value_counts / len(df)) * 100

    # Create DataFrame with both counts and percentages
    value_counts_df = pd.DataFrame(
        {"count": value_counts, column: value_counts.index, "percentage": percentages}
    ).reset_index(drop=True)

    ax = sns.barplot(
        data=value_counts_df,
        x="count",
        y=column,
        hue=column,
        orient="h",
        order=value_counts_df[column],
        palette=[color for _ in range(len(value_counts_df))],
        legend=False,
    )

    # Add labels with count and optionally percentage
    for i, v in enumerate(value_counts_df["count"]):
        if show_percentages:
            label = f"{int(v)} ({percentages[i]:.1f}%)"
        else:
            label = f"{int(v)}"
        ax.text(v + 5, i, label, va="center")  # padding

    # Extend x-axis to accommodate labels
    max_val = value_counts_df["count"].max()
    ax.set_xlim(0, max_val * extra_space)

    plt.title(title)
    plt.xlabel("Quantidade")
    plt.ylabel("")
    plt.tight_layout()
    plt.show()

In [632]:
make_barh_plot(df, "natureza", "Natureza dos projetos", palette[0], extra_space=1.18)

make_barh_plot(df, "situacao", "Situação dos projetos", palette[1], extra_space=1.14)

make_barh_plot(df, "especie", "Espécies de projetos", palette[2])

---

Resumo das variáveis categóricas:

-   **BIM**: Apenas 3.5% dos projetos usou BIM, enquanto 66.2% não usou.
-   **Natureza dos projetos**: A grande maioria dos itens (73%) é classificado como obra, e não como projeto ou projeto de investimento em infraestrutura.
-   **Situação dos projetos**: 76% projetos estão cadastrados, com 11% em execução, 8% concluídos e 3% inativados, cancelados, paralisados ou inacabados.
-   **Espécies de projetos**: 44% construção de novos projetos e 54% reforma, ampliação ou recuperação de projetos existentes.


#### 3.3.2 Variáveis quantitativas


In [633]:
count = len(df["qdtEmpregosGerados"])
nans = df["qdtEmpregosGerados"].isna().sum()
print(f"NaN percentage for qdtEmpregosGerados: {nans/count*100:.1f}%")

count = len(df["populacaoBeneficiada"])
nans = df["populacaoBeneficiada"].isna().sum()
print(f"NaN percentage for populacaoBeneficiada: {nans/count*100:.1f}%")

both_nan = df[df["qdtEmpregosGerados"].isna() & df["populacaoBeneficiada"].isna()]
print(f"Number of rows where both values are NaN: {len(both_nan)/len(df)*100:.1f}%")
print()
print(
    f'Non-nan values in both columns: {len(df[df["qdtEmpregosGerados"].notna() & df["populacaoBeneficiada"].notna()])}'
)
print(
    f'Non-nan values in qdtEmpregosGerados: {len(df[df["qdtEmpregosGerados"].notna()])}'
)
print(
    f'Non-nan values in populacaoBeneficiada: {len(df[df["populacaoBeneficiada"].notna()])}'
)

In [634]:
# quant_df = df[df["qdtEmpregosGerados"].notna() & df["populacaoBeneficiada"].notna()]

# make_barh_plot(
#     quant_df, "natureza", "Natureza dos projetos", palette[0], show_percentages=False
# )

# make_barh_plot(
#     quant_df, "situacao", "Situacao de projetos", palette[1], show_percentages=False
# )

# make_barh_plot(
#     quant_df, "especie", "Especies de projetos", palette[2], show_percentages=False
# )

Devemos considerar que estes dados estão faltando em 95% dos registros. Apesar disso, os 5% dos projetos com dados quantitativos parecem possuir a mesma distribuição da amostra geral (conferme célula anterior, removida por simplicidade) -- não são apenas projetos em execução/concluídos, como eu imaginava. Portanto, isso sugere que é uma questão de problemas de coleta de dados, e não de que esses dados só existem quando um projeto é executado ou concluído.


In [635]:
# swarm plot escolhido em vez de histograma pois só há 35 observações
plt.figure(figsize=(10, 3))
sns.swarmplot(data=df, x="qdtEmpregosGerados", size=8, color=palette[0])
plt.title("Quantidade de Empregos Gerados")
plt.xlabel("Empregos Gerados")
plt.show()

In [636]:
# outlier de empregos gerados
df[["idUnico", "nome", "descricao", "qdtEmpregosGerados"]].sort_values(
    by="qdtEmpregosGerados", ascending=False
).head(1)

In [637]:
# pq tem 8 projetos com exatamente 50 empregos gerados?
qtd_mode = df["qdtEmpregosGerados"].mode().values[0]
print(f"Mode value: {qtd_mode}")
print(
    f"Number of projects with mode value: {len(df[df['qdtEmpregosGerados'] == qtd_mode])}"
)

In [638]:
plt.figure(figsize=(10, 3))
sns.swarmplot(data=df, x="populacaoBeneficiada", size=8, color=palette[1])
plt.title("População Beneficiada")
plt.xlabel("População Beneficiada")
plt.ylabel("Projetos")
plt.show()

In [639]:
# distribuição melhor visualizada com escala logaritmica
plt.figure(figsize=(10, 3))
sns.swarmplot(
    data=df, x="populacaoBeneficiada", size=8, color=palette[1], log_scale=True
)
plt.title("População Beneficiada (escala logarítmica)")
plt.xlabel("População Beneficiada (log)")
plt.ylabel("Projetos")
plt.show()

In [640]:
# Outliers de população beneficiada
df[["idUnico", "nome", "populacaoBeneficiada"]].sort_values(
    by="populacaoBeneficiada", ascending=False
).head(4)

##### Investimento


In [641]:
# Plot normal scale
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
sns.histplot(
    data=df[~df["isInvestimentoSimbolico"]],
    x="investimentoTotal",
    bins=25,
    color=palette[0],
)
plt.xlabel("Faixa de Investimento (R$)")
plt.ylabel("Projetos")
plt.title("Projetos por Investimento")

# Plot log scale
plt.subplot(1, 2, 2)
sns.histplot(
    data=df[~df["isInvestimentoSimbolico"]],
    x="investimentoTotal",
    bins=25,
    color=palette[0],
    log_scale=True,
)
plt.xlabel("Faixa de Investimento (R$)")
plt.ylabel("Projetos")
plt.title("Projetos por Investimento (escala logarítmica)")

plt.tight_layout()
plt.show()

In [642]:
df[~df["isInvestimentoSimbolico"]]["investimentoTotal"].describe().apply(
    lambda x: f"{x:,.2f}"
)

In [643]:
def calculate_iqr(data: pd.Series) -> float:
    q75 = data.quantile(0.75)
    q25 = data.quantile(0.25)
    return q75 - q25


valores = df[~df["isInvestimentoSimbolico"]]["investimentoTotal"]
iqr = calculate_iqr(valores)
print(f"IQR: R$ {iqr:,.2f}")

Após selecionar apenas of projetos com investimentos significativos, vemos uma distribuição aproximadamente log-normal (comum com este tipo de dado não-negativo, como dinheiro). Isso significa que grandes investimento são exponencialmente mais raros que os casos de investimento menor.

Isso também significa que a média é pouco informativa sobre os dados; podemos usar mediana e IQR para ter uma ideia melhor

Ela apresenta, aproximadamente:

-   Variação entre 4 mil e 800 milhões
-   Mediana (valor que divide os dados ao meio): 2 milhões
-   IQR (amplitude interquartil): 8 milhões
-   Cerca de 50% dos projetos entre 1 e 8 milhões.

Assim, isso também indica que os valores mais altos não são outliers, mas são parte da distribuição log-normal.


In [644]:
# valores altos
df[["descricao", "investimentoTotal"]].sort_values(
    by="investimentoTotal", ascending=False
).head(10)

Várias creches parecem receber investimentos com valor simbólico.

Valores mais altos de investimento incluem:

-   SISFRON (Sistema Integrado de Monitoramento de Fronteiras): R$ 840 milhões
-   Obras rodoviárias R$ 596, R$ 359, R$ 353 milhões
-   Asessoramentos diversos: 193, 119, 112 milhões
-   Instalação de Estações Metereológicas: 118 milhões

Como só há 5% de não-nulos nas outras categorias quantitativas, opto por não calcular correlação e etc.


In [645]:
# valores baixos simbólicos
df[["nome", "investimentoTotal"]].sort_values(
    by="investimentoTotal", ascending=True
).head()

In [646]:
# valores baixos não-simbólicos
df[~df["isInvestimentoSimbolico"]][["nome", "investimentoTotal"]].sort_values(
    by="investimentoTotal", ascending=True
).head()

### Datas


In [647]:
date_cols = [
    "dataInicialPrevista",
    "dataFinalPrevista",
    "dataInicialEfetiva",
    "dataFinalEfetiva",
    "dataCadastro",
    "dataSituacao",
]

df[date_cols].info()

In [648]:
# Plot normal scale
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
sns.histplot(
    data=df["duracaoPrevista"].value_counts().reset_index(),
    bins=50,
    color=palette[0],
)
plt.xlabel("Duração Prevista (dias)")
plt.ylabel("Projetos")
plt.title("Projetos por Duração Prevista")

# Plot log scale
plt.subplot(1, 3, 2)
sns.histplot(
    data=df["duracaoPrevista"].value_counts().reset_index(),
    bins=50,
    color=palette[0],
    log_scale=True,
)
plt.xlabel("Duração Prevista (dias)")
plt.ylabel("Projetos")
plt.title("Projetos por Duração Prevista (escala logarítmica)")

# Plot log-log scale
plt.subplot(1, 3, 3)
sns.scatterplot(
    data=df["duracaoPrevista"].value_counts().reset_index(), alpha=0.5, color=palette[0]
)
plt.yscale("log")
plt.xscale("log")
plt.xlabel("Duração Prevista (dias)")
plt.ylabel("Número de Projetos")
plt.title("Projetos por Duração Prevista (escala log-log)")

plt.tight_layout()
plt.show()

In [649]:
duration_counts = df["duracaoPrevista"].dt.days.value_counts().head(5)
for days, count in duration_counts.items():
    print(f"{days} days ({days/365:.2f} years): {count} projects")

In [650]:
df["duracaoPrevista"].dt.days.divide(365).describe().apply(lambda x: f"{x:,.2f}")

In [651]:
print("IQR:", calculate_iqr(df["duracaoPrevista"].dt.days.divide(365)))

Nesse caso, parece ser uma distribuição power-law (observada como linha aproximadamente reta no gráfico log-log). Isso significa que (ainda mais do que os investimentos) a grande maioria dos projetos possui duração curta, e alguns duração extremamente longa.

Assim, é difícil dizer que haja um caso que caia 'fora da curva', pois não há um caso padrão como haveria numa distribuição normal.

Dito disso, não é incomum que projetos sejam declarados com duração exata de 1, 2, 3, 4, ou 5 anos, conforme indicado na aparente 'quantização' dos valores. Há alguns projetos declarados com duração de 364 dias em vez de 365

-   valor mínimo de 0 dias
-   valor máximo de 13 anos
-   mediana: 1.2 anos
-   IQR de 2.18 anos


In [652]:
# valores altos
df[["descricao", "duracaoPrevista", "investimentoFaixa"]].sort_values(
    by="duracaoPrevista", ascending=False
).head()

In [653]:
# valores baixos
df[["descricao", "duracaoPrevista", "investimentoFaixa"]].sort_values(
    by="duracaoPrevista", ascending=True
).head()

anomalias: projetos com investimento entre 1 e 10 milhões com duração prevista de zero dias.


In [654]:
# Check if dataInicialPrevista is in the future
future_starts = df[df["dataInicialPrevista"] > pd.Timestamp.now()]
print(f"Number of projects with start date in the future: {len(future_starts)}")

Devido à falta de dados para as datas reais, podemos comparar apenas as datas previstas.

Apesar de apenas 81 (11%) projetos estarem em situação 'em execução', apenas 4 projetos possuem início no futuro -- com a maioria em situação de 'cadastrado'. A documentação dos dados não entra em detalhes sobre o significado destas categorias.


In [655]:
# Create subplots for date distributions
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 12), sharex=True)

ax1.hist(df["dataInicialPrevista"].dropna(), bins=50)
ax1.set_title("Distribuição das Datas Iniciais Previstas")
ax1.set_ylabel("Frequência")

ax2.hist(df["dataFinalPrevista"].dropna(), bins=50)
ax2.set_title("Distribuição das Datas Finais Previstas")
ax2.set_xlabel("Data")
ax2.set_ylabel("Frequência")

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

Houve um pico de projetos entre 2012 e 2016. Conforme esperado, a maior parte termina me menos de 2 anos


In [656]:
# check big and small date values
df[["descricao", "dataFinalPrevista"]].sort_values(
    by="dataFinalPrevista", ascending=False
).head()

In [657]:
# check big and small date values
df[["descricao", "dataFinalPrevista"]].sort_values(
    by="dataFinalPrevista", ascending=True
).head()

In [658]:
# check big and small date values
df[["descricao", "dataInicialPrevista"]].sort_values(
    by="dataInicialPrevista", ascending=False
).head()

In [659]:
# check big and small date values
df[["descricao", "dataInicialPrevista"]].sort_values(
    by="dataInicialPrevista", ascending=True
).head()

In [660]:
# Preparar dados
data_for_reg = df[~df["isInvestimentoSimbolico"]].copy()
data_for_reg = data_for_reg[
    ["idUnico", "descricao", "duracaoPrevista", "investimentoTotal"]
].dropna()
data_for_reg["duracaoPrevista_days"] = data_for_reg["duracaoPrevista"].dt.days.astype(
    int
)

# Transformações apropriadas para cada distribuição
data_for_reg["investimentoTotal_log"] = np.log(
    data_for_reg["investimentoTotal"]
)  # Log-normal
data_for_reg["duracaoPrevista_log"] = np.log(
    data_for_reg["duracaoPrevista_days"]
)  # Power-law também usa log

# Criar figura com múltiplos subplots para comparação
fig, axes = plt.subplots(1, 2, figsize=(15, 8))

# 1. Escala original (não recomendado, mas para referência)
sns.regplot(
    data=data_for_reg,
    x="duracaoPrevista_days",
    y="investimentoTotal",
    scatter_kws={"alpha": 0.5},
    line_kws={"color": "red"},
    ax=axes[0],
)
axes[0].set_title("Escala Original (não ideal)")
axes[0].set_xlabel("Duração Prevista (dias)")
axes[0].set_ylabel("Investimento Total (R$)")

# 2. Log-log (melhor para capturar ambas as distribuições)
sns.regplot(
    data=data_for_reg,
    x="duracaoPrevista_log",
    y="investimentoTotal_log",
    scatter_kws={"alpha": 0.5},
    line_kws={"color": "red"},
    ax=axes[1],
)
axes[1].set_title("Log-Log (RECOMENDADO)")
axes[1].set_xlabel("Log da Duração Prevista (dias)")
axes[1].set_ylabel("Log do Investimento Total (R$)")

plt.tight_layout()
plt.show()

In [661]:
data_for_corr = df[~df["isInvestimentoSimbolico"]].copy()
data_for_corr = data_for_corr[["duracaoPrevista", "investimentoTotal"]].dropna()
data_for_corr["duracaoPrevista_days"] = data_for_corr["duracaoPrevista"].dt.days.astype(
    int
)


pg.corr(
    data_for_corr["duracaoPrevista_days"],
    data_for_corr["investimentoTotal"],
    method="spearman",  # nao parametrico
)

In [662]:
data_for_reg = data_for_reg.replace([np.inf, -np.inf], np.nan)
data_for_reg = data_for_reg.dropna()

X = sm.add_constant(data_for_reg["duracaoPrevista_log"])
y = data_for_reg["investimentoTotal_log"]
model = sm.OLS(y, X).fit()

data_for_reg["residuos"] = model.resid
data_for_reg["residuos_abs"] = np.abs(model.resid)

data_for_reg.nlargest(10, "residuos_abs").assign(
    duracaoPrevista_years=lambda x: x["duracaoPrevista_days"] / 365,
    investimentoTotal_k=lambda x: x["investimentoTotal"].apply(lambda x: f"{x:,.2f}"),
)[
    ["idUnico", "descricao", "duracaoPrevista_years", "investimentoTotal_k", "residuos"]
].round(
    {"duracaoPrevista_years": 1, "investimentoTotal_k": 0, "residuos": 1}
)

Considerando apenas os projetos com investimento significativo, podemos ver que há uma correlação entre a duração prevista e o investimento total (spearman = 0.64). Isso é esperado, pois projetos maiores tendem a durar mais. Mas com esses dados podemos calcular os projetos mais anômalos: curtos que custam muito ou demorados que custam pouco. Isso é feito usando os residuos de uma regressão linear. Com isso encontramos os itens mais anômalos, enumerados acima

Com tudo isso, encontramos os registros mais anômalos:

| ID          | Descrição                                         | Duração  | Investimento      |
| ----------- | ------------------------------------------------- | -------- | ----------------- |
| 46882.53-02 | CONSTRUÇÃO DE UNIDADE DE ATENÇÃO ESPECIALIZADA... | 1 dia    | R$ 2.493.000,00   |
| 43724.53-06 | Contratação de empresa do ramo de engenharia/a... | 3 anos   | R$ 4.684,45       |
| 557.53-69   | Projeto Estratégico do SISFRON...                 | 1.6 anos | R$ 839.664.954,32 |
| 10613.53-07 | Prestação de serviços de disponibilização de a... | 4.9 anos | R$ 47.700,00      |
| 5223.53-75  | ELABORAÇÃO DE PROJETOS BÁSICO E EXECUTIVO DE P... | 2.2 anos | R$ 20.305,41      |
| 39020.52-43 | execução das obras de adequação de capacidade,... | 1.7 anos | R$ 352.994.609,29 |
| 28451.53-40 | ADEQUACAO DE TRECHO RODOVIARIO - ENTRONCAMENTO... | 2.1 anos | R$ 359.130.057,13 |


### NLP


Podemos usar as colunas que contem texto para analisar padrões semânticos nos dados. Aqui, usamos embeddings com redução de dimensionalidade e clusterização.

Como o dataset é pequeno, uma outra alternativa seria concatenar todos os textos e usar uma LLM para fazer análises diversas. Mas não implemente isso.

O modelo de embedding se beneficia de ser usado em um computador com GPU. Assim, as células seguintes estão comentadas. A produção dos embeddings foi feita na nuvem e salvos em um arquivo que é aberto aqui.

Poderíamos continuar automatizando isso com uma abordagem como a do BERTopic, fazendo topic modelling e usando LLMs para nomear os tópicos.


In [663]:
# df[['idUnico', 'textoTotal']].to_parquet('data/projetos_sem_vetores.parquet')

In [664]:
# # fazer isso aqui usando uma GPU - por exemplo, no google colab


# def get_embeddings(df: pd.DataFrame, model: SentenceTransformer) -> pd.DataFrame:
#     # https://huggingface.co/google/embeddinggemma-300m

#     # requires HF_TOKEN env variable
#     from dotenv import load_dotenv
#     load_dotenv()


#     # gated model: only unlocks if you sign the terms and conditions consent form in the huggingface model page
#     from sentence_transformers import SentenceTransformer
#     model = SentenceTransformer("google/embeddinggemma-300m")

#     sentences = df["textoTotal"].tolist()

#     embeddings = model.encode(
#         sentences=sentences,
#         prompt_name="Clustering", # gemma specific
#         show_progress_bar=True,
#         convert_to_numpy=True,
#     )

#     sentences_with_embeddings = pd.DataFrame(
#         {"texto": sentences, "embeddings": embeddings.tolist()}
#     )
#     df_with_embeddings = pd.merge(
#         df, sentences_with_embeddings, left_on="textoTotal", right_on="texto"
#     )
#     df_with_embeddings = df_with_embeddings.drop("texto", axis=1)
#     return df_with_embeddings


# textos_df = pd.read_parquet('data/projetos_sem_vetores.parquet')
# df_with_embeddings = get_embeddings(textos_df, model)
# df_with_embeddings.to_parquet('data/projetos_com_vetores.parquet')

In [665]:
embeddings = pd.read_parquet("data/projetos_com_vetores.parquet")
df = df.merge(embeddings.drop("textoTotal", axis=1), on="idUnico", how="left")

In [666]:
def umap_df(
    df: pd.DataFrame, vectors_col: str, dimension_names: list[str]
) -> pd.DataFrame:
    # Based on https://umap-learn.readthedocs.io/en/latest/

    reducer = umap.UMAP(
        n_components=2,
        n_neighbors=15,  # Default from documentation
        min_dist=0.1,  # Default from documentation
        metric="euclidean",  # Default metric
        random_state=42,
        verbose=False,
    )

    umap_components = reducer.fit_transform(df[vectors_col].tolist())
    umap_df = pd.DataFrame(umap_components, columns=dimension_names)  # type: ignore
    return pd.concat([df, umap_df], axis=1)


def hdbscan_df(
    df: pd.DataFrame, dimension_names: list[str], min_cluster_size: int
) -> pd.DataFrame:
    # https://github.com/scikit-learn-contrib/hdbscan

    clusterer = hdbscan.HDBSCAN(min_cluster_size)
    clusters = clusterer.fit_predict(df[dimension_names])
    df["cluster"] = clusters
    df["cluster"] = df["cluster"].astype(str)

    return df


def plot_embeddings(df: pd.DataFrame, dimension_names: list[str], hue: str) -> None:
    plt.figure(figsize=(12, 8))

    sns.scatterplot(
        data=df,
        x=dimension_names[0],
        y=dimension_names[1],
        hue=hue,
        palette="tab10",
        s=100,  # marker size
        alpha=0.7,
        edgecolor="white",
        linewidth=0.5,
    )

    plt.title("UMAP Projection of Project Embeddings", fontsize=16, pad=20)
    plt.xlabel("UMAP Dimension 1", fontsize=12)
    plt.ylabel("UMAP Dimension 2", fontsize=12)
    plt.legend(title="Cluster", bbox_to_anchor=(1.05, 1), loc="upper left")
    plt.tight_layout()
    plt.show()


def plot_embeddings_interactive(
    df: pd.DataFrame, dimension_names: list[str], hue: str
) -> alt.Chart:
    chart = (
        alt.Chart(df)
        .mark_circle(size=100, opacity=0.7, strokeWidth=0.5, stroke="white")
        .encode(
            x=alt.X(dimension_names[0], title="UMAP1"),
            y=alt.Y(dimension_names[1], title="UMAP2"),
            color=alt.Color(
                hue,
                scale=alt.Scale(scheme="tableau10"),
                legend=alt.Legend(title="Cluster"),
            ),
            tooltip=[hue],
        )
        .properties(
            width=600, height=500, title="UMAP Projection of Project Embeddings"
        )
        .interactive()
    )  # Enables pan and zoom

    return chart

In [ ]:
dimension_names = ["UMAP_1", "UMAP_2"]

df_nlp_vis = df.copy()
df_nlp_vis = df_nlp_vis["duracaoPrevista"].dt.days  # altair breaks otherwise
df_nlp_vis = umap_df(df_nlp_vis, "embeddings", dimension_names)
df_nlp_vis = hdbscan_df(df_nlp_vis, dimension_names, min_cluster_size=20)
# plot_embeddings(df_nlp_vis, dimension_names, hue="cluster")

In [667]:
plot_embeddings_interactive(df_nlp_vis, dimension_names, hue="cluster")

In [ ]:
# Print examples from each cluster
print("\nExample projects from each cluster:")
for cluster in sorted(df_nlp_vis["cluster"].unique()):
    cluster_df = df_nlp_vis[df_nlp_vis["cluster"] == cluster].copy()
    print(f"\nCluster {cluster}:")
    # Show 3 random examples from each cluster
    examples = cluster_df.sample(n=min(3, len(cluster_df)))
    for _, row in examples.iterrows():
        print(f"- {row['idUnico']} - {row['nome']}")

Vemos aqui algumas das categorias que descobrimos ao analisar o texto usando embeddings. Como o gráfico é interativo, é possível clicar nos itens e investigar. No entanto, apenas olhando para os títulos de alguns exemplares, conforme a célula acima, podemos apreender algumas coisas, como:


### GIS


In [ ]:
total = len(df)
missing = df["cep"].isna().sum()
print(f"Missing CEP values: {missing} out of {total} ({missing/total:.1%})")

In [ ]:
from geocode_ceps import cep_to_coords_viacep, cep_to_coords_ipedf

ceps = df["cep"].dropna().unique()

# non-idempotent
# cep_to_coords_viacep(ceps.tolist(), "data/cep_coords_viacep.json")
# cep_to_coords_ipedf(ceps.tolist(), "data/cep_coords_ipedf.json")

In [ ]:
# avoids pandas assuming cep is a date
with open("data/cep_coords_viacep.json") as f:
    data = json.load(f)
df_viacep = pd.DataFrame.from_dict(data, orient="index")
df_viacep.columns = ["latitude", "longitude"]
print(
    f"Null records: {df_viacep['latitude'].isna().sum()} out of {len(df_viacep)} ({df_viacep['latitude'].isna().sum()/len(df_viacep):.1%})"
)

In [ ]:
# avoids pandas assuming cep is a date
with open("data/cep_coords_ipedf.json") as f:
    data = json.load(f)
df_ipedf = pd.DataFrame.from_dict(data, orient="index")
df_ipedf.columns = ["latitude", "longitude"]
print(
    f"Null records: {df_ipedf['latitude'].isna().sum()} out of {len(df_ipedf)} ({df_ipedf['latitude'].isna().sum()/len(df_ipedf):.1%})"
)

In [ ]:
df = df.merge(
    df_viacep, left_on="cep", right_index=True, how="left", suffixes=("", "_viacep")
)

df = df.merge(
    df_ipedf, left_on="cep", right_index=True, how="left", suffixes=("", "_ipedf")
)

df = df.rename(
    columns={
        "latitude": "lat_viacep",
        "longitude": "lon_viacep",
        "latitude_ipedf": "lat_ipedf",
        "longitude_ipedf": "lon_ipedf",
    }
)

In [ ]:
print(
    f"Records with coordinates from both sources: {df[['lon_viacep', 'lon_ipedf']].notna().all(axis=1).sum()}"
)

In [ ]:
# Count records with and without coordinates
has_viacep = df[["lat_viacep", "lon_viacep"]].notna().all(axis=1)
has_ipedf = df[["lat_ipedf", "lon_ipedf"]].notna().all(axis=1)

print(
    f"Records with coordinates from either source: {(has_viacep | has_ipedf).sum()} ({(has_viacep | has_ipedf).sum()/len(df):.1%})"
)

Usando APIs oficiais do governo, elas parecem ter dificuldade de usar os CEPs para obter as coordenadas dos locais. Ambos os metodos só encontraram cerca de metade dos 88 CEPs presentes dos dados.


Apenas 18% de todos os registros possui representação neste mapa. Esta amostra representa algum tipo de viés de seleção -- primeiro dos registros com CEP e depois dos CEPs com coordenadas. Dos 88 CEPs válidos, as ferramentas de geocodificação do governo só encontraram coordenadas para 41 e 33, respectivamente; 29 foram encontrados por ambas -- o que não significa que eles encontraram as mesmas coordenadas (e.g., CEP CEP 71205050).

Quando tem duas coordenadas foi optado pelas do IPEDF por obter o CEP direto, enquanto o ViaCEP apenas converte em endereço, que é convertido em coordenadas via Nominatim.


In [ ]:
import folium

# Create map centered on DF
m = folium.Map(location=[-15.7942, -47.8822], zoom_start=10)

# Define single color for all markers
COLOR = "#412355"  # Purple (LabLivre palette)

# Prepare data with proper coordinate selection
df_map = df[
    [
        "cep",
        "nome",
        "descricao",
        "lat_ipedf",
        "lon_ipedf",
        "lat_viacep",
        "lon_viacep",
        "investimentoTotal",
    ]
].copy()

# Add markers for each project with coordinates
for idx, row in df_map.iterrows():
    lat_ipedf = row["lat_ipedf"]
    lon_ipedf = row["lon_ipedf"]
    lat_viacep = row["lat_viacep"]
    lon_viacep = row["lon_viacep"]

    # Determine which coordinates to use (prioritize IPEDF)
    if pd.notna(lat_ipedf) and pd.notna(lon_ipedf):
        lat, lon = lat_ipedf, lon_ipedf
        source = "IPEDF"
    elif pd.notna(lat_viacep) and pd.notna(lon_viacep):
        lat, lon = lat_viacep, lon_viacep
        source = "ViaCEP"
    else:
        # No coordinates available
        continue

    # Prepare popup text
    popup_text = (
        f"""
    <b>CEP:</b> {row['cep']}<br>
    <b>Fonte:</b> {source}<br>
    <b>Nome:</b> {row['nome'][:50]}...<br>
    <b>Investimento:</b> R$ {row['investimentoTotal']:,.2f}
    """
        if pd.notna(row["investimentoTotal"])
        else f"""
    <b>CEP:</b> {row['cep']}<br>
    <b>Fonte:</b> {source}<br>
    <b>Nome:</b> {row['nome'][:50]}...
    """
    )

    # Add marker with radius scaled by investment amount
    folium.CircleMarker(
        location=[lat, lon],
        radius=(
            np.sqrt(row["investimentoTotal"] / 100000)
            if pd.notna(row["investimentoTotal"])
            else 5
        ),
        color=COLOR,
        fill=True,
        fillColor=COLOR,
        fillOpacity=0.7,
        popup=folium.Popup(popup_text, max_width=300),
        tooltip=f"{row['cep']} - {source}",
    ).add_to(m)

m

Podemos ver uma concentração de investimentos na área da asa sul. Há investimentos em ceilandia, gama, planaltina. de resto, há poucos ou nenhum (e.g. são sebastião).

Parte da ausência no mapa pode ser devido ao processo de seleção dos dados ou a disponibilidade dos CEPs nas APIs de geocoding.


## Relatório


Vários campos tem problema sistemáticos. Talvez o formulário seja confuso, talvez falte masking, talvez o campo não se aplique a um projeto. Isso precisaria ser investigado melhor. Para detalhes, ver seção `3.2 Qualidade`.

Em termos de

Os investimentos revelaram uma categoria implícita de investimento simbólico.

Atenção aos maiores investimentos também é relevante.


## Conclusão
